# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
import re
#
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
#
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
%matplotlib inline

[nltk_data] Downloading package punkt to C:\Users\Jorge
[nltk_data]     Assis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Jorge
[nltk_data]     Assis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jorge Assis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_query('SELECT * FROM mess', con=engine)
X = df.iloc[:,1]
y = df[df.columns[4:]]
categories_name = y.columns
categories_name

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [3]:
#df.head()

In [4]:
#df.info()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10,min_samples_leaf=1,max_features='auto',n_jobs=-1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# We split data into train and test sets. However, to get fast result on experimenting with parameters, we will subsample 5,000 sample from the training dataset to train the pipeline.
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_train, y_train, train_size = 5000)

#pipeline.fit(X_train, y_train)
pipeline.fit(X_train_sub, y_train_sub)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000017905CFCB80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10,
                                                                        n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
#y_pred = pipeline.predict(X_test)

y_pred_sub = pipeline.predict(X_test_sub)
y_pred_sub

y_test_np = y_test.values

# Report the f1 score, precision and recall for each output category of the dataset.
print(classification_report(y_test_sub, y_pred_sub,target_names=y_test.keys()))


                        precision    recall  f1-score   support

               related       0.81      0.93      0.87     12188
               request       0.84      0.30      0.44      2695
                 offer       0.00      0.00      0.00        73
           aid_related       0.75      0.46      0.57      6656
          medical_help       0.59      0.07      0.12      1249
      medical_products       0.84      0.04      0.07       792
     search_and_rescue       0.33      0.01      0.02       414
              security       0.00      0.00      0.00       264
              military       0.52      0.02      0.05       523
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.19      0.31      1001
                  food       0.83      0.27      0.41      1805
               shelter       0.84      0.10      0.18      1443
              clothing       0.00      0.00      0.00       252
                 money       1.00      

C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packa

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_leaf': [5, 10, 20],
    'clf__estimator__max_features': [0.5, 1, "log2"]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train_sub, y_train_sub)
y_pred_grid_sub = cv.predict(X_test_sub)

print(classification_report(y_test_sub, y_pred_grid_sub,target_names=y_test.keys()))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87     12188
               request       0.77      0.52      0.62      2695
                 offer       0.00      0.00      0.00        73
           aid_related       0.71      0.62      0.67      6656
          medical_help       0.53      0.29      0.38      1249
      medical_products       0.71      0.28      0.40       792
     search_and_rescue       0.65      0.18      0.28       414
              security       0.22      0.01      0.01       264
              military       0.53      0.24      0.33       523
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.64      0.69      1001
                  food       0.79      0.77      0.78      1805
               shelter       0.78      0.57      0.66      1443
              clothing       0.76      0.53      0.63       252
                 money       0.47      

C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packa

In [11]:
chosen_model = cv.best_params_

chosen_model

{'clf__estimator__max_features': 0.5,
 'clf__estimator__min_samples_leaf': 5,
 'clf__estimator__n_estimators': 200}

In [12]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,min_samples_leaf=10,max_features=0.5,n_jobs=-1)))
    ])

pipeline2.fit(X_train, y_train)

y_pred = pipeline2.predict(X_test)

print(classification_report(y_test, y_pred,target_names=y_test.keys()))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      3970
               request       0.77      0.54      0.63       948
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.63      0.68      2186
          medical_help       0.56      0.28      0.37       436
      medical_products       0.75      0.29      0.42       279
     search_and_rescue       0.71      0.14      0.24       169
              security       0.00      0.00      0.00       100
              military       0.52      0.35      0.42       166
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.70      0.74       350
                  food       0.79      0.79      0.79       593
               shelter       0.74      0.64      0.69       463
              clothing       0.68      0.56      0.61        84
                 money       0.70      

C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jorge Assis\anaconda3\lib\site-packa

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
# Save a dictionary into a pickle file.
import pickle

with open('model.pkl', 'wb') as model_file:
  pickle.dump(pipeline2, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.